# Chain of Thought Prompt Engineering

References: 
- https://medium.com/nlplanet/two-minutes-nlp-making-large-language-models-reason-with-chain-of-thought-prompting-401fd3c964d0
- https://arxiv.org/pdf/2201.11903.pdf

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
import json

sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [ ]:
# TODO: Deploy falcon endpoint with sagemaker jumpstart

In [ ]:
#endpoint_name = 'huggingface-pytorch-inference-2023-03-09-02-00-08-813'

In [ ]:
parameters = {
    "early_stopping": True,
    "length_penalty": 2.0,
    "max_new_tokens": 50,
    "temperature": .1,
    "min_length": 10,
    "no_repeat_ngram_size": 3,
}

## Mathematical Reasoning 
### Zero-shot Prompting

In [ ]:
payload = """QUESTION: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can have 3 tennis balls. How many tennis balls does he have now?
ANSWER: The answer is 11.

QUESTION: John takes care of 10 dogs. Each dog takes .5 hours a day to walk and take care of their business. How many hours a week does he spend taking care of dogs?
ANSWER:

"""

## expected output:
# # [{'generated_text': 'The answer is 7 hours a day because 10 x.5 = 7 hours. He spends 7 hours taking care of dogs a week because 7 x 7 = 56 hours.'}]

In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

### With Chain of Thought Prompting

In [ ]:
payload = """QUESTION: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
ANSWER: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

QUESTION: John takes care of 10 dogs. Each dog takes .5 hours a day to walk and take care of their business. How many hours a week does he spend taking care of dogs?
ANSWER:


"""

# # [{'generated_text': '"He spends 10 *.5 = 5 hours a day taking care of dogs. So he spends 5 * 7 = 35 hours / week taking care dogs. The answer is 35."'}]

In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

## Advanced Mathematical Reasoning - with chain of thought prompting

In [ ]:
payload = """QUESTION: Ducks need to eat 3.5 pounds of insects each week to survive. If there is a flock of ten ducks, how many pounds of insects do they need per day?
ANSWER: Ducks need 3.5 pounds of insects each week. If there is a flock of 10 ducks, then they need 3.5 x 10 = 35 pounds of insects each week. If they need 35 pounds of insects each week, then they need 35 / 7 = 5 pounds of insects each day. The answer is 5. 

QUESTION: It takes Matthew 3 minutes to dig a small hole for shrubs and 10 minutes to dig a large hole for trees. How many hours will it take him to dig 30 small holes and 15 large holes?
ANSWER: It takes Matthew 3 minutes to dig a small hole and 10 minutes to dig a large hole. So, it takes Matthew 3 x 30 = 90 minutes to dig 30 small holes. It takes Matthew 10 x 15 = 150 minutes to dig 15 large holes. So, it takes Matthew 90 + 150 = 240 minutes to dig 30 small holes and 15 large holes. 240 minutes is 4 hours. The answer is 4 hours. 

QUESTION: I have 10 liters of orange drink that are two-thirds water and I wish to add it to 15 liters of pineapple drink that is three-fifths water. But as I pour it, I spill one liter of the orange drink. How much water is in the remaining 24 liters?
ANSWER:

"""

# # [{'generated_text': '"The orange drink is 10 x 2 / 3 = 8 liters of water. The pineapple drink is 15 x 3 / 5 = 12 liter of water in it. The total water in the orange and pineapple drinks is 8"'}]


In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

As you can see in the above example for complex mathematical reasoning the models might not give you the right predicted output. 
The correnct answer is: 

"The orange drink is 10liters, 1 liter was dropped, remaining drink has 9 * 2/3 = 6 liters of water. The pineapple drink is 15 x 3 / 5 = 9 liter of water in it. The total water in the orange and pineapple drinks is 15"

## Symbolic Reasoning
For symbolic reasoning, consider the tasks of last letter concatenation, reverse list, and coin flip shown in the next image.

### Zero shot prompting

### Last Letter Concatenation

In [ ]:
payload = """QUESTION: Take the last letters of the words in "Elon Musk" and con-catenate them.
ANSWER: 

"""

# # [{"generated_text":"musk elon n"}]


In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

### With Chain of thought prompting

In [ ]:
payload = """QUESTION: Take the last letters of the words in "Elon Musk" and con-catenate them.
ANSWER: The last letter of "Elon" is "n". The last letter of "Musk" is "k'. Concatenating them is "nk". So the answer is nk.

QUESTION: Take the last letters of the words in "Mani Khanuja" and con-catenate them.
ANSWER: The last letter of "Mani" is "i". The last letter of "Khanuja" is "a". Concatenating them is "ia". So the answer is ia. 

QUESTION: Take the last letters of the words in "John Doe" and con-catenate them.
ANSWER:

"""

# #[{"generated_text":"Doe. So the answer is Doe"}]


In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

### Reverse List

### Zero shot prompting

In [ ]:
payload = """QUESTION: Reverse the sequence "glasses, pen, alarm, license".
ANSWER: 

"""

# # [{"generated_text":"license, alarm, pen, glasses, reversed"}]


In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

### With Chain of Thought prompting

In [ ]:
payload = """QUESTION: Reverse the sequence "glasses, pen, alarm, license".
ANSWER: First is glasses. Second is pen. Third is alarm. Fourth is license. Now to reverse, change the order to: Fourth is license.
Third is alarm. Second is pen. First is glasses. So the answer is
"license, alarm, pen, glasses".

QUESTION: Reverse the sequence "telephone, clock, board, spectacles".
ANSWER: First is telephone. Second is clock. Third is board. Fourth is spectacles. Now to reverse, change the order to: Fourth is spectacles.
Third is board. Second is clock. First is telephone. So the answer is
"spectacles, board, clock, telephone".

QUESTION: Reverse the sequence "cup, plate, food, fruits".
ANSWER:

"""

# # [{"generated_text":"fruits, food, plate, cup\\" is correct."}]'


In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

### Coin Flip
### Zero shot prompting

In [ ]:
payload = """QUESTION:  A coin is heads up. John does not flip the coin. Shalonda does not flip the coin. Is the coin still heads up?
ANSWER: 

"""

## [{"generated_text":"yes.... and it will remain heads up."}]

In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

In [ ]:
payload = """Q: A coin is heads up. Maybelle flips the coin. Shalonda does not flip the coin. Is the coin still heads up?
A: The coin was flipped by Maybelle. So the coin was flipped 1 time, which is an odd number. The coin started heads up, so after an odd number of flips, it will be tails up. So the answer
is no.

QUESTION:  A coin is heads up. John does not flip the coin. Shalonda does not flip the coin. Is the coin still heads up?
ANSWER:

"""

## '[{"generated_text":"The coin is still heads up because John and Shalonda did not flip the coin. So the answer is yes."}]'

In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")